In [4]:
import os
import gurobipy as gp

print("GRB_LICENSE_FILE:", os.getenv("GRB_LICENSE_FILE", "not set"))
try:
    model = gp.Model()
    print("Gurobi license is valid.")
except gp.GurobiError as e:
    print("Gurobi error:", e)

GRB_LICENSE_FILE: not set
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2656123
Academic license 2656123 - for non-commercial use only - registered to mi___@uky.edu
Gurobi license is valid.


In [5]:
import pyvista as pv
import numpy as np
height, width = 2400, 3600
# 读取 .vtp 文件
mesh_ori = pv.read("uv_mesh.vtp")
mesh_ori_u = np.fromfile("uf.dat", dtype=np.float32).reshape((height, width))
mesh_ori_v = np.fromfile("vf.dat", dtype=np.float32).reshape((height, width))

# # 查看信息
# print(mesh)
# print("Number of points:", mesh.n_points)
# print("Number of cells:", mesh.n_cells)
# print("Available point data arrays:", mesh.point_data.keys())
# print("Available cell data arrays:", mesh.cell_data.keys())
mesh_sz3 = pv.read("uv_mesh_sz3.vtp")
mesh_sz3_u = np.fromfile("uf.sz3.out", dtype=np.float32).reshape((height, width))
mesh_sz3_v = np.fromfile("vf.sz3.out", dtype=np.float32).reshape((height, width))

In [3]:
def find_true_critical_cells(u, v, epsilon=1e-6, residual_tol=1e-6):
    """
    输入:
        u, v: shape=(H, W) 的 2D 数组，表示向量场
    输出:
        critical_cell_ids: 满足条件的三角形 ID（0-based），总数为 (H-1)*(W-1)*2 个
    """
    H, W = u.shape
    n_cells = (H - 1) * (W - 1) * 2

    # 构造所有三角形的三组向量
    def build_triangle_uvs(u, v):
        # 点索引（左上角）
        v0 = np.stack([u[:-1, :-1], v[:-1, :-1]], axis=-1).reshape(-1, 2)
        v1 = np.stack([u[:-1, 1:],  v[:-1, 1:]],  axis=-1).reshape(-1, 2)
        v2 = np.stack([u[1:, :-1], v[1:, :-1]],  axis=-1).reshape(-1, 2)
        v3 = np.stack([u[1:, 1:],  v[1:, 1:]],   axis=-1).reshape(-1, 2)

        # 上三角：v0, v1, v3
        tri1 = np.stack([v0, v1, v3], axis=1)  # shape (N, 3, 2)
        # 下三角：v0, v3, v2
        tri2 = np.stack([v0, v3, v2], axis=1)

        return np.concatenate([tri1, tri2], axis=0)

    all_uvs = build_triangle_uvs(u, v)  # shape (n_cells, 3, 2)
    v0, v1, v2 = all_uvs[:, 0], all_uvs[:, 1], all_uvs[:, 2]

    # # 过滤掉任意一个为 0 的
    # zero_mask = (
    #     (np.linalg.norm(v0, axis=1) == 0.0) |
    #     (np.linalg.norm(v1, axis=1) == 0.0) |
    #     (np.linalg.norm(v2, axis=1) == 0.0)
    # )

    # v0, v1, v2 = v0[~zero_mask], v1[~zero_mask], v2[~zero_mask]
    # valid_ids = np.where(~zero_mask)[0]

    A = np.stack([v0 - v2, v1 - v2], axis=2)
    b = -v2

    # 判定矩阵是否可逆
    det = A[:, 0, 0]*A[:, 1, 1] - A[:, 0, 1]*A[:, 1, 0]
    nonsingular = np.abs(det) > 0
    A, b = A[nonsingular], b[nonsingular]
    v0, v1, v2 = v0[nonsingular], v1[nonsingular], v2[nonsingular]
    valid_ids = np.where(nonsingular)[0]

    λ = np.linalg.solve(A, b)
    λ0, λ1 = λ[:, 0], λ[:, 1]
    λ2 = 1 - λ0 - λ1

    inside = (
        (λ0 >= 0) & (λ0 <= 1) &
        (λ1 >= 0) & (λ1 <= 1) &
        (λ2 >= 0) & (λ2 <= 1)
    )

    # v_interp = λ0[:, None] * v0 + λ1[:, None] * v1 + λ2[:, None] * v2
    # residual = np.linalg.norm(v_interp, axis=1)
    # close_to_zero = residual < residual_tol

    # final_mask = inside & close_to_zero
    final_mask = inside 
    return valid_ids[final_mask]  # 返回所有 true critical cell 的 flat ID

def build_critical_bitmap(u, v):
    """
    输入:
        u, v: shape=(H, W) 的 2D numpy 数组，表示向量场
    输出:
        bitmap: shape=(H-1, W-1, 2)，每个格子中两个三角形是否含 critical point
    """
    H, W = u.shape
    n_cells = (H - 1) * (W - 1) * 2

    # 获取所有的 critical triangle ids
    critical_ids = find_true_critical_cells(u, v)

    # 初始化 bitmap
    bitmap = np.zeros((H - 1, W - 1, 2), dtype=np.uint8)

    for cid in critical_ids:
        grid_cell_id = cid // 2
        tri_idx = cid % 2  # 0: upper, 1: lower
        row = grid_cell_id // (W - 1)
        col = grid_cell_id % (W - 1)
        bitmap[row, col, tri_idx] = 1

    return bitmap



In [6]:
# 考虑退化的情况
def find_true_critical_cells(u, v, epsilon=1e-6, residual_tol=1e-6):
    H, W = u.shape
    def build_triangle_uvs(u, v):
        v0 = np.stack([u[:-1, :-1], v[:-1, :-1]], axis=-1).reshape(-1, 2)
        v1 = np.stack([u[:-1, 1:],  v[:-1, 1:]],  axis=-1).reshape(-1, 2)
        v2 = np.stack([u[1:, :-1], v[1:, :-1]],  axis=-1).reshape(-1, 2)
        v3 = np.stack([u[1:, 1:],  v[1:, 1:]],   axis=-1).reshape(-1, 2)
        tri1 = np.stack([v0, v1, v3], axis=1)
        tri2 = np.stack([v0, v3, v2], axis=1)
        return np.concatenate([tri1, tri2], axis=0)

    all_uvs = build_triangle_uvs(u, v)
    v0, v1, v2 = all_uvs[:, 0], all_uvs[:, 1], all_uvs[:, 2]

    # 判断是否 degenerate（任意一个点为 (0,0)）
    degenerate_mask = (
        ((v0[:, 0] == 0.0) & (v0[:, 1] == 0.0)) |
        ((v1[:, 0] == 0.0) & (v1[:, 1] == 0.0)) |
        ((v2[:, 0] == 0.0) & (v2[:, 1] == 0.0))
    )

    A = np.stack([v0 - v2, v1 - v2], axis=2)
    b = -v2
    det = A[:, 0, 0]*A[:, 1, 1] - A[:, 0, 1]*A[:, 1, 0]
    nonsingular = np.abs(det) > 0

    computable = ~degenerate_mask & nonsingular
    A, b = A[computable], b[computable]
    v0, v1, v2 = v0[computable], v1[computable], v2[computable]
    valid_ids = np.where(computable)[0]

    λ = np.linalg.solve(A, b)
    λ0, λ1 = λ[:, 0], λ[:, 1]
    λ2 = 1 - λ0 - λ1
    inside = (
        (λ0 >= 0) & (λ0 <= 1) &
        (λ1 >= 0) & (λ1 <= 1) &
        (λ2 >= 0) & (λ2 <= 1)
    )
    critical_ids = valid_ids[inside]
    return critical_ids, degenerate_mask

def build_critical_bitmap(u, v):
    H, W = u.shape
    n_cells = (H - 1) * (W - 1) * 2

    critical_ids, degenerate_mask = find_true_critical_cells(u, v)

    # 初始化为 0
    bitmap = np.zeros((H - 1, W - 1, 2), dtype=np.int8)

    # 标记 degenerate 的 triangle 为 -1
    for tri_id in np.where(degenerate_mask)[0]:
        grid_cell_id = tri_id // 2
        k = tri_id % 2
        i = grid_cell_id // (W - 1)
        j = grid_cell_id % (W - 1)
        bitmap[i, j, k] = -1

    # 标记真实 critical point 为 1
    for tri_id in critical_ids:
        grid_cell_id = tri_id // 2
        k = tri_id % 2
        i = grid_cell_id // (W - 1)
        j = grid_cell_id % (W - 1)
        bitmap[i, j, k] = 1

    return bitmap

In [7]:
# critical_cells = find_true_critical_cells(mesh_ori_u, mesh_ori_v)
# print(f"检测到 {len(critical_cells)} 个包含 critical point 的三角形 in original mesh")
# critical_cells_sz3 = find_true_critical_cells(mesh_sz3_u, mesh_sz3_v)
# print(f"检测到 {len(critical_cells_sz3)} 个包含 critical point 的三角形 in sz3 mesh")

bitmap_ori = build_critical_bitmap(mesh_ori_u, mesh_ori_v)
# 统计
print("ori critical =", np.sum(bitmap_ori == 1))
print("ori degenerate =", np.sum(bitmap_ori == -1))
print("ori non-critical =", np.sum(bitmap_ori == 0))

# total_cells = (3600 - 1) * (2400 - 1) * 2
# count_neg1 = np.sum(bitmap_ori == -1)
# count_0 = np.sum(bitmap_ori == 0)
# count_1 = np.sum(bitmap_ori == 1)

# print("Total:", total_cells)
# print("-1:", count_neg1)
# print(" 0:", count_0)
# print(" 1:", count_1)
# print("Sum:", count_neg1 + count_0 + count_1)

bitmap_sz3 = build_critical_bitmap(mesh_sz3_u, mesh_sz3_v)
# 统计
print("sz3 critical =", np.sum(bitmap_sz3 == 1))
print("sz3 degenerate =", np.sum(bitmap_sz3 == -1))
print("sz3 non-critical =", np.sum(bitmap_sz3 == 0))
# bitmap_sz3_hard_copy = bitmap_sz3.copy()

dec1ori0 = np.where((bitmap_sz3 == 1) & (bitmap_ori == 0))
print("bitmap_dec == 1 and bitmap_ori == 0的三角形数量:", len(dec1ori0[0]))

ori critical = 20938
ori degenerate = 5735860
ori non-critical = 11511204
sz3 critical = 91648
sz3 degenerate = 7201
sz3 non-critical = 17169153
bitmap_dec == 1 and bitmap_ori == 0的三角形数量: 3393


In [8]:
import numpy as np
import gurobipy as gp
from concurrent.futures import ProcessPoolExecutor

NUM_THREADS = 128  # You can adjust this as needed

def compute_lambda_numpy(u_vals, v_vals):
    A = np.array([[2, 1], [1, 2]])
    b = np.array([u_vals[0] - u_vals[1], v_vals[0] - v_vals[1]])
    x, y = np.linalg.solve(A, b)
    return np.array([1 - x - y, x, y])

def is_outside(lambda_vec):
    return np.any(lambda_vec < 0) or np.any(lambda_vec > 1)

def solve_single_triangle_gurobi(u_vals, v_vals, epsilon, tau=1e-4, M=10.0):
    model = gp.Model()
    model.setParam("OutputFlag", 0)
    u_vars = [model.addVar(lb=ui - epsilon, ub=ui + epsilon) for ui in u_vals]
    v_vars = [model.addVar(lb=vi - epsilon, ub=vi + epsilon) for vi in v_vals]
    model.update()

    du = u_vars[0] - u_vars[1]
    dv = v_vars[0] - v_vars[1]
    x = (2 * du - dv) / 3
    y = (-du + 2 * dv) / 3
    lambda_vec = [1 - x - y, x, y]
    triangle_z_vars = []
    for k in range(3):
        z_low = model.addVar(vtype=gp.GRB.BINARY)
        z_high = model.addVar(vtype=gp.GRB.BINARY)
        triangle_z_vars += [z_low, z_high]
        model.addConstr(lambda_vec[k] <= -0.001 + M * (1 - z_low))
        model.addConstr(lambda_vec[k] >= 1 + 0.001 - M * (1 - z_high))
        # model.addConstr(z_low + z_high <= 1)
        model.addConstr(z_low + z_high == 1) # 这里是咋改的？
    model.addConstr(gp.quicksum(triangle_z_vars) >= 1)

    abs_terms = []

    for idx in range(3):
        diff_u = u_vars[idx] - u_vals[idx]
        abs_u = model.addVar(lb=0.0)
        model.addConstr(abs_u >= diff_u)
        model.addConstr(abs_u >= -diff_u)
        abs_terms.append(abs_u)

        diff_v = v_vars[idx] - v_vals[idx]
        abs_v = model.addVar(lb=0.0)
        model.addConstr(abs_v >= diff_v)
        model.addConstr(abs_v >= -diff_v)
        abs_terms.append(abs_v)

    model.setObjective(gp.quicksum(abs_terms), gp.GRB.MINIMIZE)

    model.optimize()
    if model.status == gp.GRB.OPTIMAL:
        u_opt = np.array([var.X for var in u_vars])
        v_opt = np.array([var.X for var in v_vars])
        return u_opt, v_opt
    return None, None

def solve_patch(i, j, u_dec, v_dec, bitmap_ori, bitmap_dec, patch_size, epsilon, tau=1e-4, M=10.0):
    # print(f"[Thread] Starting patch ({i},{j})")
    H, W = u_dec.shape
    i_end = min(i + patch_size, H - 1)
    j_end = min(j + patch_size, W - 1)

    model = gp.Model()
    model.setParam("OutputFlag", 0)
    patch_u = {}
    patch_v = {}
    triangle_list = []

    # 添加ghost layer
    i_start = max(i - 1, 0)
    j_start = max(j - 1, 0)
    i_end = min(i + patch_size + 1, H - 1)
    j_end = min(j + patch_size + 1, W - 1)

    for ii in range(i, i_end + 1):
        for jj in range(j, j_end + 1):
            # patch_u[(ii, jj)] = model.addVar(lb=u_dec[ii, jj] - epsilon, ub=u_dec[ii, jj] + epsilon)
            # patch_v[(ii, jj)] = model.addVar(lb=v_dec[ii, jj] - epsilon, ub=v_dec[ii, jj] + epsilon)

            # 添加boundary的约束，使得不参与优化
            is_boundary = (ii == 0 or ii == H - 1 or jj == 0 or jj == W - 1)
            if is_boundary:
                patch_u[(ii, jj)] = u_dec[ii, jj]  # use constant
                patch_v[(ii, jj)] = v_dec[ii, jj]
            else:
                patch_u[(ii, jj)] = model.addVar(lb=u_dec[ii, jj] - epsilon, ub=u_dec[ii, jj] + epsilon)
                patch_v[(ii, jj)] = model.addVar(lb=v_dec[ii, jj] - epsilon, ub=v_dec[ii, jj] + epsilon)

    model.update()

    for ii in range(i, i_end):
        for jj in range(j, j_end):
            for tri_id in range(2):
                if not (bitmap_ori[ii, jj, tri_id] == 0 and bitmap_dec[ii, jj, tri_id] == 1):
                    continue
                if tri_id == 0:
                    vtx = [(ii, jj), (ii + 1, jj), (ii + 1, jj + 1)]
                else:
                    vtx = [(ii, jj), (ii, jj + 1), (ii + 1, jj + 1)]
                triangle_list.append(vtx)
                u_vars = [patch_u[v] for v in vtx]
                v_vars = [patch_v[v] for v in vtx]
                du = u_vars[0] - u_vars[1]
                dv = v_vars[0] - v_vars[1]
                x = (2 * du - dv) / 3
                y = (-du + 2 * dv) / 3
                lambda_vec = [1 - x - y, x, y]
                triangle_z_vars = []
                for k in range(3):
                    z_low = model.addVar(vtype=gp.GRB.BINARY)
                    z_high = model.addVar(vtype=gp.GRB.BINARY)
                    triangle_z_vars += [z_low, z_high]
                    model.addConstr(lambda_vec[k] <= -0.001 + M * (1 - z_low))
                    model.addConstr(lambda_vec[k] >= 1 + 0.001 - M * (1 - z_high))
                    model.addConstr(z_low + z_high <= 1) #至少一个lambda 出界
                    # model.addConstr(z_low + z_high == 1) # 这里是咋改的？
                model.addConstr(gp.quicksum(triangle_z_vars) >= 1)

    abs_terms = []

    for (ii, jj), var in patch_u.items():
        if isinstance(var, gp.Var):
            diff_u = patch_u[(ii, jj)] - u_dec[ii, jj]
            abs_u = model.addVar(lb=0.0)
            model.addConstr(abs_u >= diff_u)
            model.addConstr(abs_u >= -diff_u)
            abs_terms.append(abs_u)

    for (ii, jj), var in patch_v.items():
        if isinstance(var, gp.Var):
            diff_v = patch_v[(ii, jj)] - v_dec[ii, jj]
            abs_v = model.addVar(lb=0.0)
            model.addConstr(abs_v >= diff_v)
            model.addConstr(abs_v >= -diff_v)
            abs_terms.append(abs_v)

    model.setObjective(gp.quicksum(abs_terms), gp.GRB.MINIMIZE)

    try:
        model.optimize()
        if model.status == gp.GRB.OPTIMAL:
            print(f"[Thread] Patch ({i},{j}) solved successfully")
            u_patch = np.zeros_like(u_dec[i:i_end+1, j:j_end+1])
            v_patch = np.zeros_like(v_dec[i:i_end+1, j:j_end+1])
            for (ii, jj), var in patch_u.items():
                u_patch[ii - i, jj - j] = var.X if isinstance(var, gp.Var) else var
            for (ii, jj), var in patch_v.items():
                v_patch[ii - i, jj - j] = var.X if isinstance(var, gp.Var) else var
            return (i, j), u_patch, v_patch, triangle_list, None
    # except Exception as e:
    #     print(f"  [Error] Gurobi solver failed at patch ({i},{j}): {e}")
    # print(f"[Thread] Patch ({i},{j}) failed, triggering fallback")
    # return (i, j), None, None, triangle_list, (u_dec[i:i_end+1, j:j_end+1], v_dec[i:i_end+1, j:j_end+1])
        else:
            raise RuntimeError(f"Patch ({i},{j}) MILP not solved optimally: status = {model.status}")
    except Exception as e:
        print(f"[Error] Patch ({i},{j}) failed due to: {e}")
        raise RuntimeError(f"Fallback not allowed. Patch ({i},{j}) terminated.")

def patchwise_milp_fix_parallel(u_dec, v_dec, bitmap_ori, bitmap_dec, patch_size=50, epsilon=0.05, tau=1e-4, M=10.0):
    H, W = u_dec.shape
    u_fixed = u_dec.copy()
    v_fixed = v_dec.copy()

    tasks = []
    for i in range(0, H - 1, patch_size):
        for j in range(0, W - 1, patch_size):
            tasks.append((i, j, u_dec, v_dec, bitmap_ori, bitmap_dec, patch_size, epsilon, tau, M))

    with ProcessPoolExecutor(max_workers=NUM_THREADS) as executor:
        futures = [executor.submit(solve_patch, *args) for args in tasks]
        for f in futures:
            (i, j), u_patch, v_patch, triangle_list, fallback_data = f.result()
            if u_patch is not None:
                for ii in range(u_patch.shape[0]):
                    for jj in range(u_patch.shape[1]):
                        u_fixed[i + ii, j + jj] = u_patch[ii, jj]
                        v_fixed[i + ii, j + jj] = v_patch[ii, jj]
                for vtx in triangle_list:
                    u_vals = np.array([u_fixed[ii, jj] for (ii, jj) in vtx])
                    v_vals = np.array([v_fixed[ii, jj] for (ii, jj) in vtx])
                    lambda_fixed = compute_lambda_numpy(u_vals, v_vals)
                    if not is_outside(lambda_fixed):
                        lambda_before = compute_lambda_numpy(
                            np.array([u_dec[ii, jj] for (ii, jj) in vtx]),
                            np.array([v_dec[ii, jj] for (ii, jj) in vtx])
                        )
                        print(f"  [Verify Failed] Triangle {vtx} after patch MILP is still inside")
                        print(f"    lambda before fix: {lambda_before}")
                        print(f"    lambda after  fix: {lambda_fixed}")
                        print(f"    u_dec: {[u_dec[ii, jj] for (ii, jj) in vtx]}")
                        print(f"    v_dec: {[v_dec[ii, jj] for (ii, jj) in vtx]}")
                        print(f"    u_fixed: {[u_fixed[ii, jj] for (ii, jj) in vtx]}")
                        print(f"    v_fixed: {[v_fixed[ii, jj] for (ii, jj) in vtx]}")
            elif fallback_data is not None:
                print(f"[Fallback] Solving triangle-wise for patch ({i},{j})")
                u_blk, v_blk = fallback_data
                for ii in range(u_blk.shape[0] - 1):
                    for jj in range(v_blk.shape[1] - 1):
                        for tri_id in range(2):
                            if not (bitmap_ori[i + ii, j + jj, tri_id] == 0 and bitmap_dec[i + ii, j + jj, tri_id] == 1):
                                continue
                            if tri_id == 0:
                                vtx = [(i + ii, j + jj), (i + ii + 1, j + jj), (i + ii + 1, j + jj + 1)]
                            else:
                                vtx = [(i + ii, j + jj), (i + ii, j + jj + 1), (i + ii + 1, j + jj + 1)]
                            u_vals = np.array([u_dec[ii, jj] for (ii, jj) in vtx])
                            v_vals = np.array([v_dec[ii, jj] for (ii, jj) in vtx])
                            u_opt, v_opt = solve_single_triangle_gurobi(u_vals, v_vals, epsilon, tau, M)
                            if u_opt is not None:
                                for idx in range(3):
                                    ii_, jj_ = vtx[idx]
                                    u_fixed[ii_, jj_] = u_opt[idx]
                                    v_fixed[ii_, jj_] = v_opt[idx]

    return u_fixed, v_fixed


In [9]:
u_fixed, v_fixed = patchwise_milp_fix_parallel(mesh_sz3_u, mesh_sz3_v, bitmap_ori, bitmap_sz3, patch_size=250, epsilon=0.2, tau=1e-4, M=100.0)

[Thread] Patch (0,3500) solved successfully
[Thread] Patch (250,3500) solved successfully
[Thread] Patch (0,2500) solved successfully
[Thread] Patch (500,3500) solved successfully
[Thread] Patch (0,750) solved successfully
[Thread] Patch (0,0) solved successfully
[Thread] Patch (0,500) solved successfully
[Thread] Patch (0,250) solved successfully
[Thread] Patch (0,1000) solved successfully
[Thread] Patch (0,3250) solved successfully
[Thread] Patch (0,1250) solved successfully
[Thread] Patch (0,1500) solved successfully
[Thread] Patch (0,1750) solved successfully
[Thread] Patch (0,2250) solved successfully
[Thread] Patch (0,2000) solved successfully
[Thread] Patch (750,3500) solved successfully
[Thread] Patch (0,2750) solved successfully
[Thread] Patch (0,3000) solved successfully
[Thread] Patch (250,2500) solved successfully
[Thread] Patch (250,0) solved successfully
[Thread] Patch (250,250) solved successfully
[Thread] Patch (250,500) solved successfully
[Thread] Patch (250,750) solv

In [18]:
# 在u_fixed和v_fixed上只检查bitmap_dec ==1 and bitmap_ori == 0的三角形
dec1ori0 = np.where((bitmap_sz3 == 1) & (bitmap_ori == 0))
print("bitmap_dec == 1 and bitmap_ori == 0的三角形数量:", len(dec1ori0[0]))

def verify_lambda_outside(u_fixed, v_fixed,u_ori,v_ori,u_dec,v_dec, bitmap_ori, bitmap_dec, tau=1e-6):
    H, W = u_fixed.shape
    total_checked = 0
    failed = 0

    def is_outside(lambda_vec):
        return np.any(lambda_vec < 0) or np.any(lambda_vec > 1 )

    for ii in range(H - 1):
        for jj in range(W - 1):
            for tri_id in range(2):
                if (bitmap_ori[ii, jj, tri_id] == 0 and bitmap_dec[ii, jj, tri_id] == 1):
                    if tri_id == 0:
                        vtx = [(ii, jj), (ii + 1, jj), (ii + 1, jj + 1)]
                    else:
                        vtx = [(ii, jj), (ii, jj + 1), (ii + 1, jj + 1)]

                    u_vals = np.array([u_fixed[i, j] for (i, j) in vtx])
                    v_vals = np.array([v_fixed[i, j] for (i, j) in vtx])
                    lambda_vec = compute_lambda_numpy(u_vals, v_vals)

                    total_checked += 1
                    if not is_outside(lambda_vec):
                        failed += 1
                        print(f"[Validation Fail] Triangle {vtx} has lambda still inside: {lambda_vec}")
                        print(f"     lambda ori: {compute_lambda_numpy([u_ori[i, j] for (i, j) in vtx], [v_ori[i, j] for (i, j) in vtx])}")
                        print(f"     lambda dec: {compute_lambda_numpy([u_dec[i, j] for (i, j) in vtx], [v_dec[i, j] for (i, j) in vtx])}")

    print(f"\n[Validation Summary]")
    print(f"  Total triangles checked: {total_checked}")
    print(f"  Triangles failed (still inside): {failed}")
    print(f"  Pass rate: {(1 - failed / total_checked) * 100:.2f}%")

verify_lambda_outside(u_fixed, v_fixed,mesh_ori_u,mesh_ori_v,mesh_sz3_u,mesh_sz3_v, bitmap_ori, bitmap_sz3, tau=1e-6)

bitmap_dec == 1 and bitmap_ori == 0的三角形数量: 3393
[Validation Fail] Triangle [(1499, 3266), (1500, 3266), (1500, 3267)] has lambda still inside: [0.32198461 0.61037763 0.06763776]
     lambda ori: [0.32750845 0.42854071 0.24395084]
     lambda dec: [0.32198461 0.61037763 0.06763776]
[Validation Fail] Triangle [(2000, 2999), (2000, 3000), (2001, 3000)] has lambda still inside: [0.56299039 0.00627991 0.43072971]
     lambda ori: [0.57795175 0.04269846 0.37934979]
     lambda dec: [0.56299039 0.00627991 0.43072971]

[Validation Summary]
  Total triangles checked: 3393
  Triangles failed (still inside): 2
  Pass rate: 99.94%


In [12]:
def print_uv_and_lambda_for_dec1_ori0(u_ori, v_ori, u_dec, v_dec, bitmap_ori, bitmap_dec, tau=1e-6, max_print=10):
    H, W = u_ori.shape
    count = 0

    idx = np.where((bitmap_dec == 1) & (bitmap_ori == 0))
    print("bitmap_dec == 1 and bitmap_ori == 0 的三角形数量:", len(idx[0]))

    def get_vtx(ii, jj, tri_id):
        return [(ii, jj), (ii + 1, jj), (ii + 1, jj + 1)] if tri_id == 0 else [(ii, jj), (ii, jj + 1), (ii + 1, jj + 1)]

    for ii, jj, tri_id in zip(*idx):
        if count >= max_print:
            break

        vtx = get_vtx(ii, jj, tri_id)
        u_ori_vals = np.array([u_ori[i, j] for (i, j) in vtx])
        v_ori_vals = np.array([v_ori[i, j] for (i, j) in vtx])
        u_dec_vals = np.array([u_dec[i, j] for (i, j) in vtx])
        v_dec_vals = np.array([v_dec[i, j] for (i, j) in vtx])
        lambda_ori = compute_lambda_numpy(u_ori_vals, v_ori_vals)
        lambda_dec = compute_lambda_numpy(u_dec_vals, v_dec_vals)

        print(f"\n[Triangle {vtx}]")
        print(f"  u_ori: {u_ori_vals}")
        print(f"  v_ori: {v_ori_vals}")
        print(f"  lambda_ori: {lambda_ori}")
        print(f"  u_dec: {u_dec_vals}")
        print(f"  v_dec: {v_dec_vals}")
        print(f"  lambda_dec: {lambda_dec}")
        count += 1

# print_uv_and_lambda_for_dec1_ori0(mesh_ori_u, mesh_ori_v, mesh_sz3_u,mesh_sz3_v, bitmap_ori, bitmap_sz3, tau=0, max_print=10)
print_uv_and_lambda_for_dec1_ori0(mesh_ori_u, mesh_ori_v,u_fixed,v_fixed, bitmap_ori, bitmap_sz3, tau=0, max_print=10)

bitmap_dec == 1 and bitmap_ori == 0 的三角形数量: 72200

[Triangle [(0, 2050), (1, 2050), (1, 2051)]]
  u_ori: [0. 0. 0.]
  v_ori: [0. 0. 0.]
  lambda_ori: [1. 0. 0.]
  u_dec: [-0.00040297  0.00069691  0.00068214]
  v_dec: [ 0.         -0.00080025 -0.00078867]
  lambda_dec: [ 1.00009987e+00 -1.00000003e-03  9.00125325e-04]

[Triangle [(1, 2084), (2, 2084), (2, 2085)]]
  u_ori: [0. 0. 0.]
  v_ori: [0. 0. 0.]
  lambda_ori: [1. 0. 0.]
  u_dec: [-0.00081116  0.00051316  0.0004917 ]
  v_dec: [-0.00035574 -0.00070711 -0.00068025]
  lambda_dec: [ 1.00032431e+00 -9.99999989e-04  6.75685238e-04]

[Triangle [(1, 2085), (1, 2086), (2, 2086)]]
  u_ori: [0. 0. 0.]
  v_ori: [0. 0. 0.]
  lambda_ori: [1. 0. 0.]
  u_dec: [0.0002483  0.00023762 0.00047048]
  v_dec: [-0.00182319 -0.00032853 -0.00065323]
  lambda_dec: [ 1.00049466e+00  5.05341062e-04 -9.99999970e-04]

[Triangle [(1, 2091), (2, 2091), (2, 2092)]]
  u_ori: [0. 0. 0.]
  v_ori: [0. 0. 0.]
  lambda_ori: [1. 0. 0.]
  u_dec: [0.00018587 0.00190209 0.0